In [130]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException, JavascriptException, NoSuchElementException, ElementNotInteractableException
import time
from time import sleep
from datetime import date, datetime, timedelta 
import numpy as np
import pandas as pd
import os
import re
from functions import allMondays, intHolidayClosures, create_trips, country_holidays

In [164]:
#CONSTANTS
URL = 'https://camper.fti.de/'
DELAY = 3
PAGE_LOAD_ID = "SearchMask"
COOKIE_ID = '//*[@id="uc-center-container"]/div[2]/div/div[1]/div/button'
COOKIE_JS_PATH = 'document.querySelector("#usercentrics-root").shadowRoot.querySelector("#uc-center-container > div.sc-gWXbKe.fSIThp > div > div.sc-cxpSdN.ihpUDX > div > button")'
NOTIFICATION_JS_PATH = 'document.querySelector("#pid-14572 > div.cleverpush-confirm.cleverpush-confirm-modal.cleverpush-confirm-windows.cleverpush-confirm-chrome.cleverpush-confirm-chrome.cleverpush-confirm-1 > div.cleverpush-confirm-wrap > div.cleverpush-clearfix.cleverpush-clearfix-bottom > div.cleverpush-confirm-buttons > button.cleverpush-confirm-btn.cleverpush-confirm-btn-deny")'
POPUP_JS_PATH = 'document.querySelector("#trboModule_12920_202268_88_container > div.trbo-closeButton.trbo-theme-0.trboType-icon > i")'
CITY = 'Los Angeles'
CITY_JS_PATH = 'document.querySelector("#SearchMask > div.flexContainer.location > div:nth-child(1) > div:nth-child(1) > div > div > input")'
CAL_ELEMS_XPATH = "//span[contains(@class, 'cell day')]"
PU_CALENDAR_JS_PATH = 'document.querySelector("#SearchMask > div:nth-child(2) > div:nth-child(1) > div > div > div:nth-child(1) > input")'
DO_CALENDAR_JS_PATH = "document.querySelector('#SearchMask > div:nth-child(2) > div:nth-child(2) > div > div > div:nth-child(1) > input')"
DAY_MONTH_BUTTON = "span.day__month_btn.up"
MONTH_YEAR_BUTTON = "span.month__year_btn.up"

DO_MONTH_JS_PATH = "document.querySelector('#SearchMask > div:nth-child(2) > div:nth-child(2) > div > div > div:nth-child(2) > header > span.day__month_btn.up')"
PU_MONTH_TEXT_CLASS_NAME = "vdp-datepicker__calendar"
DO_MONTH_TEXT_CLASS_NAME = "vdp-datepicker__calendar"
PU_MONTH_NEXT_BTN_CLASS_NAME = "span[class = 'next']"
PU_MONTH_PREV_BTN_CLASS_NAME = "span[class = 'prev']"
DO_MONTH_PREV_BTN_CLASS_NAME = "span[class = 'prev']"
DO_MONTH_TEXT_XPATH = "//*[@id='SearchMask']/div[2]/div[2]/div/div/div[2]/header/span[3]"
SUBMIT_BUTTON_ID = "SubmitButton"
LOAD_MORE_RESULTS = "document.querySelector('#ResultList > button')"
NO_RESULTS_MESSAGE = " return document.querySelector('#pid-14572 > div:nth-child(37) > div > div.mainDialog.warning > div.submitContainer > button')"
SHORT_TIMEOUT  = 1.2   # give enough time for the loading element to appear
LONG_TIMEOUT = 30  # give enough time for loading to finish
LOADING_ELEMENT_XPATH = "//*[@id='MainLoader']/div/img"

NO_RESULTS_MESSAGE_OK_BUTTON = "document.querySelector('#pid-14572 > div:nth-child(35) > div > div.mainDialog.warning > div.submitContainer > button')"
#replace 29 with 35

_CITIES = [
           #{'city':'San Francisco','state':'CA', 'abbr':'SFO', 'country':'USA'}#,
           #{'city':'Las Vegas','state':'NV', 'abbr':'LAS', 'country':'USA'} ,
          #{'city':'Los Angeles','state':'CA', 'abbr':'LAX', 'country':'USA'}
           # {'city':'Seattle','state':'WA', 'abbr':'SEA', 'country':'USA'}#,
            #{'city':'Salt Lake City','state':'UT', 'abbr':'SLC', 'country':'USA'}#,
         # {'city':'Denver','state':'CO', 'abbr':'DEN', 'country':'USA'},
         # {'city':'Chicago','state':'IL', 'abbr':'CHI', 'country':'USA'},
         #  {'city':'Dallas','state':'TX', 'abbr':'DFW', 'country':'USA'},
          {'city':'Orlando','state':'FL', 'abbr':'MCO', 'country':'USA'}
        ]


_GERMAN_MONTHS = {
    'Jan' : 'Jan',
    'Feb' : 'Feb',
    'Mar' : 'Mär',
    'Apr' : 'Apr',
    'May' : 'Mai',
    'Jun' : 'Jun',
    'Jul' : 'Jul',
    'Aug' : 'Aug',
    'Sep' : 'Sep',
    'Oct': 'Okt',
    'Nov': 'Nov',
    'Dec': 'Dez'

}


In [132]:
def human():

    time.sleep(int(np.random.rand(1)*4))

In [133]:
def create_browser(URL, ELEMENT_ID):
    
    #BOT DETECTION PREVENTION
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    #REMOVES POPUPS
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    options.add_experimental_option("prefs",prefs)

    #AVOIDS DEPTRECATION WARNING
    s=Service(ChromeDriverManager().install())
    browser = webdriver.Chrome(service=s, options = options)

    browser.get(URL)

    try:
        myElem = WebDriverWait(browser, DELAY).until(EC.presence_of_element_located((By.ID, PAGE_LOAD_ID)))
        print ("Page is ready!")
    except TimeoutException:
        print ("Couldn't find element")

    return browser


#browser = create_browser(URL, PAGE_LOAD_ID)

In [134]:
#CURRENTLY NOT REQUIRED, Interruptions 

#COOKIES

# browser.execute_script(f"return {COOKIE_JS_PATH}").click()
# #deny news

# browser.execute_script(f"return {NOTIFICATION_JS_PATH}").click()

# #close popup
# browser.execute_script(f"return {POPUP_JS_PATH}").click()

In [135]:
def select_city(city_path, city):

    try:
        browser.find_element(By.CSS_SELECTOR, "i.svg-icon.close-x").click()

    except (ElementNotInteractableException, NoSuchElementException):
        pass


    browser.find_element(By.CSS_SELECTOR, "div.location-input-wrapper").find_element(By.CSS_SELECTOR, "input.text").send_keys(f"{city}")

    human()

    browser.find_element(By.CSS_SELECTOR, "div.location-input-wrapper").find_element(By.CSS_SELECTOR, "input.text").send_keys(Keys.RETURN)

    human()




In [136]:
def select_year(start_or_end_year):
    years = browser.find_elements(By.XPATH, "//span[contains(@class, 'cell year')]")
    for year in years:

        if year.text == start_or_end_year:
            human()
            year.click()

In [137]:
def select_month(start_or_end_month):
    months = browser.find_elements(By.XPATH, "//span[contains(@class, 'cell month')]")
    for month in months:

        if month.text[:3] == start_or_end_month:
            human()
            month.click()

In [138]:
def select_day(start_or_end_day):
    cal_elems = browser.find_elements(By.XPATH, f"{CAL_ELEMS_XPATH}")
    cal_elems_dict = {}
    for ele in cal_elems:

        if ele.text.isdigit():

            cal_elems_dict[ele.text] = ele
            
    cal_elems_dict[str(start_or_end_day)].click()


In [139]:
#reset calandar
# def reset_calendar(cal_open_elem, cal_month_elem, cal_month_prev_button):
#     browser.execute_script(f"return {cal_open_elem}").click()
    
#     pickup = None

#     if cal_open_elem[71] == '1':
#         pickup = True

#     elif cal_open_elem[71] == '2':
#         pickup = False


#     if pickup == True:

#         start = 2
#         end = 5
#     else:
#         start = 0
#         end = 3

    
#     if pickup == True:

#         cal_current_year = browser.find_element(By.CLASS_NAME, f"{cal_month_elem}").text[6:10]

#     else:

#         cal_current_year = browser.execute_script(f" return {cal_month_elem}").text[4:10]
    
    
#     currentYear = str(datetime.now().year)
#     my_date = datetime.now()
#     current_month = my_date.strftime("%B")[:3]

    

#     if pickup == True:

#         curr_cal_month = browser.find_element(By.CLASS_NAME, f"{cal_month_elem}").text[start:end]

#     else:

#         curr_cal_month = browser.execute_script(f" return {cal_month_elem}").text[start:end]



#     german_current_month = _GERMAN_MONTHS[current_month]

#     while german_current_month != curr_cal_month and cal_current_year != currentYear:

#         print(f"Clicked {curr_cal_month} prev button")

#         browser.find_element(By.CSS_SELECTOR, f"{cal_month_prev_button }").click()

#         if pickup == True:

#             curr_cal_month = browser.find_element(By.CLASS_NAME, f"{cal_month_elem}").text[start:end]

#         else:

#             curr_cal_month = browser.execute_script(f" return {cal_month_elem}").text[start:end]


#         human()



#     select_day(28)


## Full Script

In [140]:
error = None

try:
    error = None
except Exception as error:
    pass

if error:
    print("it works")


In [142]:
browser = create_browser(URL, PAGE_LOAD_ID)



Page is ready!


In [146]:
if not browser.find_element(By.CLASS_NAME, "location-input-wrapper"):
    print("yes")

else:print("no")

no


In [ ]:
By.CLASS_NAME, "location-input-wrapper"

In [ ]:
EC.presence_of_element_located((By.ID, PAGE_LOAD_ID)))

In [150]:
browser.find_element(By.CLASS_NAME, "location-input-wrapper").is_displayed()

True

In [163]:
script_start = date(2022, 12, 1)
#END DATE OF LAST REQUESTED TRIP
int_stop_date = date(2023, 4, 30)
browser = create_browser(URL, PAGE_LOAD_ID)
human()
human()
human()

#accept cookies
browser.execute_script("""return document.getElementById('usercentrics-root').shadowRoot;""").find_element(By.CSS_SELECTOR, "button[role = 'button']").click()

for city in _CITIES:
    
    US = True if city['country'] == 'USA' else False

    iterator= create_trips(int_stop_date, US)
    iterator_last = enumerate(create_trips(int_stop_date, US))
    *_, last = iterator_last


    for trip_index, trip in enumerate(iterator):

        #when script breaks, this can skip already scraped results
        if pd.to_datetime(trip['start_date']).date() < script_start:
            continue



        select_city(CITY_JS_PATH, city['city'])

        print(f"Trip Mumber {trip_index}")
        print(f"Current City: {city['city']}")

        #**************************************CALENDARS**************************************
        #OPEN CALENDAR
        trip['start_month'] = _GERMAN_MONTHS[trip['start_month']]
        trip['end_month'] = _GERMAN_MONTHS[trip['end_month']]

        #ENTER INFORMATION INTO PICKUP CALENDAR
        #open pick-up calendar

        try:
            browser.execute_script(f"return {PU_CALENDAR_JS_PATH}").click()

        except ElementClickInterceptedException:
            print("A POPUP INTERRUPTED THE SCRIPT")
            human()
            browser.execute_script("""return document.getElementById('usercentrics-root').shadowRoot;""").find_element(By.CSS_SELECTOR, "button[role = 'button']").click()
            human()
            browser.execute_script(f"return {PU_CALENDAR_JS_PATH}").click()


        human()
        #click the month year button
        browser.find_element(By.CLASS_NAME, f"{PU_MONTH_TEXT_CLASS_NAME}").find_element(By.CSS_SELECTOR, f"{DAY_MONTH_BUTTON}").click()
        human()
        browser.find_element(By.CSS_SELECTOR, f"{MONTH_YEAR_BUTTON}").click()

        human()

        select_year(trip['start_year'])
        human()
        select_month(trip['start_month'])
        human()
        select_day(trip['start_day'])



        human()

        #ENTER INFORMATION INTO DROP-OFF CALENDAR
        try:
            browser.execute_script(f"return {DO_CALENDAR_JS_PATH}").click()

        except ElementClickInterceptedException:
            print("A POPUP INTERRUPTED THE SCRIPT")
            human()
            browser.execute_script("""return document.getElementById('usercentrics-root').shadowRoot;""").find_element(By.CSS_SELECTOR, "button[role = 'button']").click()
            human()
            browser.execute_script(f"return {DO_CALENDAR_JS_PATH}").click()
        
        human()
        #there is 5 vdp datepicker calendar items, right now we need the third calendar
        browser.find_elements(By.CLASS_NAME, f"{DO_MONTH_TEXT_CLASS_NAME}")[3].find_element(By.CSS_SELECTOR, f"{DAY_MONTH_BUTTON}").click()
        human()
        #fourth calendar for this one
        browser.find_elements(By.CLASS_NAME, f"{DO_MONTH_TEXT_CLASS_NAME}")[4].find_element(By.CSS_SELECTOR, f"{MONTH_YEAR_BUTTON}").click()
        human()
        select_year(trip['end_year'])
        human()
        select_month(trip['end_month'])
        human()
        select_day(trip['end_day'])

        print("CALENDAR FINISHED")

        human()
        #*******************END OF CALENDAR SELECTIONS**********************

        #CHECK FOR PAGE LOAD, IF NO RESULTS THEN IT IS STUCK ON LOADING SCREEEN, SO GO TO NEXT ITERATION
        browser.find_element(By.ID, f"{SUBMIT_BUTTON_ID }").click()
        
        try:
            #WebDriverWait(browser, SHORT_TIMEOUT
                #).until(EC.presence_of_element_located((By.XPATH, LOADING_ELEMENT_XPATH)))

            WebDriverWait(browser, LONG_TIMEOUT
                ).until_not(EC.presence_of_element_located((By.XPATH, LOADING_ELEMENT_XPATH)))

        except TimeoutException:
            print("Stuck on the Loading Screen")

            #WHAT HAPPENS WHEN STUCK ON LOADING SCREEN
            browser.close()
            browser = create_browser(URL, PAGE_LOAD_ID)
            human()
            human()
            human()
            browser.execute_script("""return document.getElementById('usercentrics-root').shadowRoot;""").find_element(By.CSS_SELECTOR, "button[role = 'button']").click()

            #re-enter the city
            select_city(CITY_JS_PATH, city['city'])

            human()
            continue
            #pass #continue in for loop

        print("LOAD ICON HAS DISAPPEARED")

        human()

        #DEAL WITH THE NO RESULTS BUTTION 
        try:
            obj = browser.find_element(By.CSS_SELECTOR, "div.mainDialog.warning").find_element(By.CSS_SELECTOR, "div.submitContainer").find_element(By.CSS_SELECTOR, "button.button.ghost.mainColor.block-btn")
        #obj = browser.execute_script(f"return {NO_RESULTS_MESSAGE_OK_BUTTON}")
        except NoSuchElementException:
            obj = None


        if obj:

            print("No Results: Going to Next Trip Iteration")
            browser.find_element(By.CSS_SELECTOR, "div.mainDialog.warning").find_element(By.CSS_SELECTOR, "div.submitContainer").find_element(By.CSS_SELECTOR, "button.button.ghost.mainColor.block-btn").click()
            
            print("Moving on to the next iteration.")


            continue

        else:
            print("Results Found")

        #SCROLL DOWN TO LOAD MORE RESULTS

        browser.execute_script("return window.scrollTo(0, document.body.scrollHeight);")
        human()
        human()

        try: 
            browser.execute_script(f"return {LOAD_MORE_RESULTS}").click()
        except AttributeError:

            sleep(60)
            print("sleepin")
            print("Load More Results Button Was Not There")

        
        #ALLOWS TIME TO LOAD MORE RESULTS IF PRESENT
        human()
        human()
        human()
    
        #SCRAPE
        LISTINGS_CLASS_NAME = "resultCard"
        listings = browser.find_elements(By.CLASS_NAME, f"{LISTINGS_CLASS_NAME}")


        trip_data = []
        for card in listings:

            vehicle_class = card.find_element(By.CLASS_NAME, "provider-data").text.split("\n")[0]
            company_name = card.find_element(By.CLASS_NAME, "providerLogo").get_attribute("alt")
            price_string = card.find_element(By.CLASS_NAME, "camper-details").find_element(By.CSS_SELECTOR, 'div.priceSummary.full > dl > dd').text.split('\n')
            total_price = '.'.join(re.findall(r'\d+', price_string[0]))
            daily_price = '.'.join(re.findall(r'\d+', price_string[1]))


            trip_data.append({
            'city': city['city'],
            'start_date':trip['start_date'],
            'end_date':trip['end_date'],
            'total_price':total_price,
            'daily_price': daily_price,
            'vehicle_class':vehicle_class,
            'company_name':company_name
            })

        #SAVE RESULTS TO FOLDER
        trip_data_df = pd.DataFrame(trip_data)

        try: 
            os.makedirs(f"../Output Data/FTI Camper/{city['city']} ")
        except FileExistsError:
            pass

        print("Saving File")
        trip_data_df.to_csv(f"../Output Data/FTI Camper/{city['city']}/{trip_index} ti {city['city']} {date.today()}.csv", index=False)

    

        #SCROLL TO TOP
        browser.execute_script("return window.scrollTo(0,0)","")

        if not browser.find_element(By.CLASS_NAME, "location-input-wrapper").is_displayed():
            browser.close()
            human()
            browser = create_browser(URL, PAGE_LOAD_ID)
            human()
            human()
            human()
            browser.execute_script("""return document.getElementById('usercentrics-root').shadowRoot;""").find_element(By.CSS_SELECTOR, "button[role = 'button']").click()


        

        human()

browser.close()


Page is ready!
Trip Mumber 4
Current City: Dallas
CALENDAR FINISHED
LOAD ICON HAS DISAPPEARED
Results Found
Saving File
Trip Mumber 5
Current City: Dallas
CALENDAR FINISHED
LOAD ICON HAS DISAPPEARED
Results Found
Saving File
Trip Mumber 6
Current City: Dallas
CALENDAR FINISHED
LOAD ICON HAS DISAPPEARED
Results Found
Saving File
Trip Mumber 7
Current City: Dallas
CALENDAR FINISHED
LOAD ICON HAS DISAPPEARED
Results Found
sleepin
Load More Results Button Was Not There
Saving File
Trip Mumber 8
Current City: Dallas
CALENDAR FINISHED
LOAD ICON HAS DISAPPEARED
Results Found
Saving File
Trip Mumber 9
Current City: Dallas
CALENDAR FINISHED
LOAD ICON HAS DISAPPEARED
Results Found
Saving File
Trip Mumber 10
Current City: Dallas
CALENDAR FINISHED
LOAD ICON HAS DISAPPEARED
Results Found
Saving File
Trip Mumber 11
Current City: Dallas
CALENDAR FINISHED
LOAD ICON HAS DISAPPEARED
Results Found
Saving File
Trip Mumber 12
Current City: Dallas
CALENDAR FINISHED
LOAD ICON HAS DISAPPEARED
Results Found
sl

### To-do

- combine csv files into one file for each city